# Time series to supervised
Before machine learning can be used, time series forecasting problems must be re-framed as supervised learning problems. From a sequence to pairs of input and output sequences as described at https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/

In [ ]:
import boto3
import io
from sagemaker import get_execution_role

role = get_execution_role()
bucket ='sagemaker-data-energy-demand'

In [ ]:
import numpy as np 
import pandas as pd 

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def series_to_supervised(data,  col_names, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
     data: Sequence of observations as a list or NumPy array.
     n_in: Number of lag observations as input (X).
     n_out: Number of observations as output (y).
     dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
     Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('%s(t-%d)' % (col_names[j], i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    if i == 0:
        names += [('%s(t)' % (col_names[j])) for j in range(n_vars)]
    else:
        names += [('%s(t+%d)' % (col_names[j], i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
CITY = 'LosAngeles'
df_file = 'dataframes/%s_selectedfeatures.csv' % CITY

df_location = 's3://{}/{}'.format(bucket, df_file)
df = pd.read_csv(df_location, index_col ='datetime')
df.head()

In [ ]:
#set the column we want to predict (demand) to the first columns for consistency
values = df.values
# ensure all data is float
values = values.astype('float32')
# frame as supervised learning
reframed = series_to_supervised(values, df.columns, 1, 1)
# drop columns we don't want to predict
reframed.drop(reframed.columns[[15,16,17,18,19,20,21,22,23,24,25,26,27]], axis=1, inplace=True)

cols = list(reframed.columns)
cols.remove('demand(t)')
cols.remove('demand(t-1)')
cols.insert(0,'demand(t)')
reframed = reframed[cols]
reframed.index = df.index[:-1]


In [ ]:
reframed.head()

In [ ]:
reframed.columns

In [ ]:
# save as csv file to continue in another notebook
csv_buffer = io.StringIO()
s3_resource = boto3.resource('s3')
key = 'dataframes/%s_reframed.csv' % CITY

reframed.to_csv(csv_buffer, compression=None)
s3_resource.Object(bucket, key).put(Body=csv_buffer.getvalue())